In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn

import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:512'

dev="cpu"
if torch.cuda.is_available():
    dev="cuda:0"
DEVICE = torch.device(dev)
print(dev)
MAX_LENGTH = 50

cpu


In [ ]:
count = 0
english_lines = []
with open("drive/MyDrive/CSCI 544 Project/Data/PhoMT/tokenization/train/train.en", encoding='utf-8') as f:
  for line in f:
    # english_lines.append("\t" + line)

    english_lines.append(line)
    count = count + 1
    if count > 50000:
      break

count = 0
vietnamese_lines = []
with open("drive/MyDrive/CSCI 544 Project/Data/PhoMT/tokenization/train/train.vi", encoding='utf-8') as f:
  for line in f:
    # vietnamese_lines.append("\t" + line)
    vietnamese_lines.append(line)
    count = count + 1
    if count > 50000:
      break

In [ ]:
print(english_lines[0])
print("--> len of english_lines is", len(english_lines))
print(vietnamese_lines[0])
print("--> len of vietnamese_lines is", len(vietnamese_lines))

It begins with a countdown .

--> len of english_lines is 50001
Câu chuyện bắt đầu với buổi lễ đếm ngược .

--> len of vietnamese_lines is 50001


#**MT5**

In [ ]:
!pip install transformers sentencepiece datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 48.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 73.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 57.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.4/269.4 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [ ]:
from transformers import AdamW, AutoModelForSeq2SeqLM, AutoTokenizer
import torch
from torch import optim
from torch.nn import functional as F
from transformers import AdamW, AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import get_linear_schedule_with_warmup
import random
from tqdm import tqdm_notebook

In [ ]:
model_repo = "google/mt5-small"
tokenizer = AutoTokenizer.from_pretrained(model_repo)
model = AutoModelForSeq2SeqLM.from_pretrained(model_repo)


/usr/local/lib/python3.9/dist-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
model = model.cuda()

In [ ]:
seq_len = 20

In [ ]:
# example_input_str = '<jp> This is just a test nbuig.'
example_input_str = "Hôm nay là một ngày rất đẹp."
input_ids = tokenizer.encode(example_input_str, return_tensors='pt')
print('Input IDs:', input_ids)

tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
print('Tokens:', tokens)

Input IDs: tensor([[  447, 19698,  8735,   796,   326,  1266,   259,  3665,   259,   286,
          1075,   355,  7105,   260,     1]])
Tokens: ['▁H', 'ôm', '▁nay', '▁là', '▁m', 'ột', '▁', 'ngày', '▁', 'r', 'ất', '▁đ', 'ẹp', '.', '</s>']


In [ ]:
def encode_input_str(text, tokenizer, seq_len):
  input_ids = tokenizer.encode(
      text = text,
      return_tensors = 'pt',
      padding = 'max_length',
      truncation = True,
      max_length = seq_len)

  return input_ids[0]
  
def encode_target_str(text, tokenizer, seq_len):
  token_ids = tokenizer.encode(
      text = text,
      return_tensors = 'pt',
      padding = 'max_length',
      truncation = True,
      max_length = seq_len)

  return token_ids[0]

def format_translation_data(input_text, target_text, tokenizer, seq_len=128):
  input_token_ids = encode_input_str(
      input_text, tokenizer, seq_len)
  
  target_token_ids = encode_target_str(
      target_text,tokenizer, seq_len)

  return input_token_ids, target_token_ids

def transform_batch(batch, tokenizer):
  inputs = []
  targets = []

  for input_text, target_text in batch:
    formatted_data = format_translation_data(input_text, target_text, tokenizer, seq_len)
    
    if formatted_data is None:
      continue
    
    input_ids, target_ids = formatted_data
    inputs.append(input_ids.unsqueeze(0))
    targets.append(target_ids.unsqueeze(0))
    
  batch_input_ids = torch.cat(inputs).cuda()
  batch_target_ids = torch.cat(targets).cuda()

  return batch_input_ids, batch_target_ids

def get_data_generator(dataset, tokenizer, batch_size=32):
  random.shuffle(dataset)
  print(dataset[0])
  for i in range(0, len(dataset), batch_size):
    raw_batch = dataset[i:i+batch_size]
    yield transform_batch(raw_batch, tokenizer)

In [ ]:
train_dataset = []
for i in range(len(english_lines)):
  train_dataset.append([vietnamese_lines[i], english_lines[i]])

# test_dataset = []
# for i in range(len(test_vietnamese_lines)):
#   test_dataset.append([test_vietnamese_lines[i], test_english_lines[i]])

NameError: ignored

In [ ]:
len(train_dataset)

50001

In [ ]:
data_gen = get_data_generator(train_dataset, tokenizer, 16)
data_batch = next(data_gen)
print('Input shape:', data_batch[0].shape)
print('Output shape:', data_batch[1].shape)

['Bây giờ , tiến về lúc này thì các bạn thấy là : 15 triệu điện thoại di động một tháng .\n', 'Now , fast - forward to today and this is what you see : the 15 million cell phones a month .\n']
Input shape: torch.Size([16, 20])
Output shape: torch.Size([16, 20])


In [ ]:
n_epochs = 8
batch_size = 16
print_freq = 100
checkpoint_freq = 500
lr = 5e-4
n_batches = int(np.ceil(len(train_dataset) / batch_size))
total_steps = n_epochs * n_batches
n_warmup_steps = int(total_steps * 0.01)

In [ ]:
# Optimizer
optimizer = AdamW(model.parameters(), lr=lr)
scheduler = get_linear_schedule_with_warmup(
    optimizer, n_warmup_steps, total_steps)

losses = []

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
model_path = 'drive/MyDrive/CSCI 544 Project/Hieu and Vu/models/mt5.pt'

In [ ]:
min_loss = 1000000
for epoch_idx in range(n_epochs):
  # Randomize data order
  data_generator = get_data_generator(train_dataset, tokenizer, batch_size)
                
  for batch_idx, (input_batch, label_batch) \
      in tqdm_notebook(enumerate(data_generator), total=n_batches):
    optimizer.zero_grad()

    # Forward pass
    model_out = model.forward(
        input_ids = input_batch,
        labels = label_batch)

    # Calculate loss and update weights
    loss = model_out.loss
    losses.append(loss.item())
    loss.backward()
    optimizer.step()
    scheduler.step()

    # Print training update info
    if (batch_idx + 1) % print_freq == 0:
      avg_loss = np.mean(losses[-print_freq:])
      print('Epoch: {} | Step: {} | Avg. loss: {:.3f} | lr: {}'.format(
          epoch_idx+1, batch_idx+1, avg_loss, scheduler.get_last_lr()[0]))
    if loss.item() < min_loss:
      min_loss = loss.item()
      torch.save(model.state_dict(), model_path)
# torch.save(model.state_dict(), model_path)

<ipython-input-18-a86608f3b0e5>:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  in tqdm_notebook(enumerate(data_generator), total=n_batches):


  0%|          | 0/3126 [00:00<?, ?it/s]

['Hãy kể chúng tôi nghe cách anh hiểu về mã nguồn mở và cách nó dẫn tới sự phát triển của Linux .\n', 'Tell us how you got to understand open source and how it lead to the development of Linux .\n']
Epoch: 1 | Step: 100 | Avg. loss: 17.035 | lr: 0.0002
Epoch: 1 | Step: 200 | Avg. loss: 6.070 | lr: 0.0004
Epoch: 1 | Step: 300 | Avg. loss: 3.573 | lr: 0.0004989902253816948
Epoch: 1 | Step: 400 | Avg. loss: 2.792 | lr: 0.0004969706761450845
Epoch: 1 | Step: 500 | Avg. loss: 2.548 | lr: 0.0004949511269084741
Epoch: 1 | Step: 600 | Avg. loss: 2.481 | lr: 0.0004929315776718636
Epoch: 1 | Step: 700 | Avg. loss: 2.442 | lr: 0.0004909120284352533
Epoch: 1 | Step: 800 | Avg. loss: 2.374 | lr: 0.0004888924791986429
Epoch: 1 | Step: 900 | Avg. loss: 2.296 | lr: 0.00048687292996203247
Epoch: 1 | Step: 1000 | Avg. loss: 2.227 | lr: 0.0004848533807254221
Epoch: 1 | Step: 1100 | Avg. loss: 2.189 | lr: 0.0004828338314888117
Epoch: 1 | Step: 1200 | Avg. loss: 2.161 | lr: 0.0004808142822522013
Epoch: 1 |

  0%|          | 0/3126 [00:00<?, ?it/s]

['Và nó khá là đơn giản nhưng cũng là khá lớn , một việc lớn đấy .\n', "And it sounds quite simple , but actually , it 's quite a big , big job .\n"]
Epoch: 2 | Step: 100 | Avg. loss: 1.799 | lr: 0.00043989821471847486
Epoch: 2 | Step: 200 | Avg. loss: 1.805 | lr: 0.00043787866548186444
Epoch: 2 | Step: 300 | Avg. loss: 1.782 | lr: 0.0004358591162452541
Epoch: 2 | Step: 400 | Avg. loss: 1.759 | lr: 0.0004338395670086437
Epoch: 2 | Step: 500 | Avg. loss: 1.762 | lr: 0.00043182001777203327
Epoch: 2 | Step: 600 | Avg. loss: 1.769 | lr: 0.00042980046853542295
Epoch: 2 | Step: 700 | Avg. loss: 1.773 | lr: 0.00042778091929881253
Epoch: 2 | Step: 800 | Avg. loss: 1.795 | lr: 0.0004257613700622021
Epoch: 2 | Step: 900 | Avg. loss: 1.766 | lr: 0.00042374182082559174
Epoch: 2 | Step: 1000 | Avg. loss: 1.772 | lr: 0.00042172227158898136
Epoch: 2 | Step: 1100 | Avg. loss: 1.796 | lr: 0.00041970272235237094
Epoch: 2 | Step: 1200 | Avg. loss: 1.779 | lr: 0.00041768317311576057
Epoch: 2 | Step: 1300 

...

['Đó không phải là điều bạn nghe , bạn biết đấy , từ báo chí Tây phương .\n', 'It is not what you hear , you know , from the Western press .\n']
Epoch: 6 | Step: 100 | Avg. loss: 0.840 | lr: 0.00018737377817271188
Epoch: 6 | Step: 200 | Avg. loss: 0.837 | lr: 0.00018535422893610145
Epoch: 6 | Step: 300 | Avg. loss: 0.877 | lr: 0.00018333467969949108
Epoch: 6 | Step: 400 | Avg. loss: 0.850 | lr: 0.00018131513046288071
Epoch: 6 | Step: 500 | Avg. loss: 0.852 | lr: 0.0001792955812262703
Epoch: 6 | Step: 600 | Avg. loss: 0.885 | lr: 0.00017727603198965992
Epoch: 6 | Step: 700 | Avg. loss: 0.855 | lr: 0.00017525648275304952
Epoch: 6 | Step: 800 | Avg. loss: 0.862 | lr: 0.00017323693351643912
Epoch: 6 | Step: 900 | Avg. loss: 0.873 | lr: 0.00017121738427982875
Epoch: 6 | Step: 1000 | Avg. loss: 0.881 | lr: 0.00016919783504321836
Epoch: 6 | Step: 1100 | Avg. loss: 0.857 | lr: 0.00016717828580660796
Epoch: 6 | Step: 1200 | Avg. loss: 0.886 | lr: 0.0001651587365699976
Epoch: 6 | Step: 1300 | Av

  0%|          | 0/3126 [00:00<?, ?it/s]

['Tôi đi bộ đến một bữa tiệc nơi những người tôi biết cố gắng để biết tôi và hỏi nơi mà tôi đang ở nhưng tôi không thể nói ra những gì tôi đã thấy\n', "I walked to a party where people I know try hard to know me and ask where I 've been , but I ca n't explain what I 've seen to them .\n"]
Epoch: 7 | Step: 100 | Avg. loss: 0.600 | lr: 0.00012424266903627112
Epoch: 7 | Step: 200 | Avg. loss: 0.578 | lr: 0.00012222311979966072
Epoch: 7 | Step: 300 | Avg. loss: 0.590 | lr: 0.00012020357056305033
Epoch: 7 | Step: 400 | Avg. loss: 0.583 | lr: 0.00011818402132643994
Epoch: 7 | Step: 500 | Avg. loss: 0.590 | lr: 0.00011616447208982956
Epoch: 7 | Step: 600 | Avg. loss: 0.588 | lr: 0.00011414492285321916
Epoch: 7 | Step: 700 | Avg. loss: 0.628 | lr: 0.00011212537361660878
Epoch: 7 | Step: 800 | Avg. loss: 0.604 | lr: 0.00011010582437999839
Epoch: 7 | Step: 900 | Avg. loss: 0.621 | lr: 0.000108086275143388
Epoch: 7 | Step: 1000 | Avg. loss: 0.597 | lr: 0.00010606672590677761
Epoch: 7 | Step: 1100

  0%|          | 0/3126 [00:00<?, ?it/s]

['Thằng bé không thể nào nuốt hay ăn được nữa .\n', 'He can no longer swallow or eat .\n']
Epoch: 8 | Step: 100 | Avg. loss: 0.378 | lr: 6.111155989983036e-05
Epoch: 8 | Step: 200 | Avg. loss: 0.363 | lr: 5.909201066321997e-05
Epoch: 8 | Step: 300 | Avg. loss: 0.376 | lr: 5.707246142660958e-05
Epoch: 8 | Step: 400 | Avg. loss: 0.371 | lr: 5.5052912189999196e-05
Epoch: 8 | Step: 500 | Avg. loss: 0.380 | lr: 5.3033362953388805e-05
Epoch: 8 | Step: 600 | Avg. loss: 0.383 | lr: 5.1013813716778414e-05
Epoch: 8 | Step: 700 | Avg. loss: 0.365 | lr: 4.899426448016803e-05
Epoch: 8 | Step: 800 | Avg. loss: 0.382 | lr: 4.697471524355764e-05
Epoch: 8 | Step: 900 | Avg. loss: 0.370 | lr: 4.495516600694725e-05
Epoch: 8 | Step: 1000 | Avg. loss: 0.373 | lr: 4.293561677033686e-05
Epoch: 8 | Step: 1100 | Avg. loss: 0.350 | lr: 4.0916067533726474e-05
Epoch: 8 | Step: 1200 | Avg. loss: 0.379 | lr: 3.889651829711609e-05
Epoch: 8 | Step: 1300 | Avg. loss: 0.369 | lr: 3.687696906050569e-05
Epoch: 8 | Step: 

Raw input text: Chúng tôi không hiểu vì sao
tensor([ 1536,  6927,  2957,   266,  1284,   382,  5140,   259,  7568, 12181,
            1,     0,     0,     0,     0,     0,     0,     0,     0,     0])
Truncated input text: Chúng tôi không hiểu vì sao</s><pad><pad><pad><pad><pad><pad><pad><pad><pad>


In [ ]:
output_tokens = model.generate(input_ids, num_beams=20, num_return_sequences=3)
# print(output_tokens)
for token_set in output_tokens:
  print(tokenizer.decode(token_set, skip_special_tokens=True))

We did not understand why we did. 
We didn't understand why we did. 
We didn't understand why we didn't do that.


In [ ]:
count = 0
test_english_lines = []
with open("drive/MyDrive/CSCI 544 Project/Data/PhoMT/tokenization/train/train.en", encoding='utf-8') as f:
  for line in f:
    # english_lines.append("\t" + line)
    count = count + 1
    if count > 50000:
       test_english_lines.append(line)
       
    if count == 70000:
      break

count = 0
test_vietnamese_lines = []
with open("drive/MyDrive/CSCI 544 Project/Data/PhoMT/tokenization/train/train.vi", encoding='utf-8') as f:
  for line in f:
    # vietnamese_lines.append("\t" + line)
    count = count + 1
    if count > 50000:
      test_vietnamese_lines.append(line)
    if count == 70000:
      break

In [ ]:
print(len(test_english_lines))

20000


In [ ]:
real_test_vietnamese_sentences = []
real_test_english_sentences = []
for i, line in enumerate(test_vietnamese_lines):
  if len(line.split(" ")) <= 20:
    real_test_vietnamese_sentences.append(line)
    real_test_english_sentences.append(test_english_lines[i])

In [ ]:
len(real_test_english_sentences)

10542

In [ ]:
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))



<All keys matched successfully>

In [ ]:

test_sentence = "Hôm nay là một ngày đẹp trời"
input_ids = encode_input_str(
      text = test_sentence,
      tokenizer = tokenizer,
      seq_len = model.config.max_length)

input_ids = input_ids.unsqueeze(0)

output_tokens = model.generate(input_ids, num_beams=20, num_return_sequences=1)
for token_set in output_tokens:
  print(tokenizer.decode(token_set, skip_special_tokens=True))

Today is a lovely day. 


In [ ]:
predictions = []
count = 0
for line in real_test_vietnamese_sentences[:20]:
  count = count + 1
  print(count)
  test_sentence = line
  input_ids = encode_input_str(
      text = test_sentence,
      tokenizer = tokenizer,
      seq_len = model.config.max_length)

  input_ids = input_ids.unsqueeze(0)

  output_tokens = model.generate(input_ids, num_beams=20, num_return_sequences=1)
  for token_set in output_tokens:
    predictions.append(tokenizer.decode(token_set, skip_special_tokens=True))

1


/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


In [ ]:
candidate_corpus = []
references_corpus = []
for i,line in enumerate(real_test_english_sentences[:20]):
  references_corpus.append([line.split()])
  candidate_corpus.append(predictions[i].split())

In [ ]:
import pandas as pd

In [ ]:
res = pd.DataFrame(list(zip(real_test_vietnamese_sentences[:20], real_test_english_sentences[:20], predictions[:20])))

In [ ]:
res

,0,1,2
0,Có rất nhiều khó khăn .\n,It was a lot of hard work .\n,There's a lot of hard work.
1,Có một chút tò mò đối với tôi .\n,That was a little curious to me .\n,There's a bit of curious about me.
2,Hãy để tôi lùi lại và xem qua tại sao thành ph...,Let me step back and take a look at why the ci...,Let me step back and take a step back and take...
3,Giống như cả một thế giới mới về thông tin cảm...,It 's like a whole new world of sensory inform...,It's like a new world of information.
4,"Không chỉ thế , cả khứu giác nữa .\n","But also , the sense of smell .\n","Not only that, emotional nectar."
5,Những gì xảy ra với tôi cũng là thành phố đang...,What struck me also was how much the city was ...,What's happened to me is what's going on.
6,"Mất thị lực , dù sao , đó là cả một câu chuyện...","Lose your sight , though , and it 's a whole o...","It's a disbelief, anyway, that's"
7,Rất nhiều thông tin tốt .\n,A lot of the information is good .\n,That's a lot of good information.
8,Rất nhiều cái lại trái ngược .\n,A lot of it 's kind of reversed .\n,There's a lot that's reversed.
9,bạn có thể tưởng tượng ra những gì chúng ám ch...,You 've got to figure out what they actually m...,You can imagine what we've been talking about.


In [ ]:
from torchtext.data.metrics import bleu_score
bleu_score(candidate_corpus, references_corpus)

0.12261816398587717